In [1]:
code = {
    "name": "commands",
    "body": [
        {
            "name": "repeat-n",
            "count": 10,
            "body": [
                {
                    "name": "gpio-write",
                    "pin": 16,
                    "value": 1
                },
                {
                    "name": "wait-sec",
                    "duration": 1
                },
                {
                    "name": "gpio-write",
                    "pin": 16,
                    "value": 0
                },
                {
                    "name": "wait-sec",
                    "duration": 1
                }
            ]
        }
    ]
}

In [2]:
from proc_comp import parser

p = parser.parse(code)

#print(p)
print(p.pprint())

SeqExp([
  RepeatExp((U32) 10, [
    SeqExp([
      ProcSetExp(gpio_mode[16], (U8) 1)
      ProcSetExp(gpio_value[16], (U8) 1)
    ])
    WaitTimeExp((U32) 1)
    SeqExp([
      ProcSetExp(gpio_mode[16], (U8) 1)
      ProcSetExp(gpio_value[16], (U8) 0)
    ])
    WaitTimeExp((U32) 1)
  ])
])


In [3]:
from proc_comp.codegen.codegen import CodeGen

G = CodeGen()
G.code_gen(p)


Procedures:
proc$1
	ProcSet gpio_mode[16] (U8) 1
	ProcSet gpio_value[16] (U8) 1
	ProcUnop time idt (UInt32) param$U32$3
	ProcBinop (UInt32) param$U32$3 + (U32) 1 (UInt32) param$U32$3
	ProcBlock time >= (UInt32) param$U32$3
	ProcSet gpio_mode[16] (U8) 1
	ProcSet gpio_value[16] (U8) 0
	ProcUnop time idt (UInt32) param$U32$4
	ProcBinop (UInt32) param$U32$4 + (U32) 1 (UInt32) param$U32$4
	ProcBlock time >= (UInt32) param$U32$4
proc$2
	ProcSet (UInt32) param$U32$1 (U32) 0
	ProcSet (UInt32) param$U32$2 (U32) 10
	ProcCall proc$1
	ProcUnop (UInt32) param$U32$1 ++ (UInt32) param$U32$1
	ProcIfElse (UInt32) param$U32$1 < (UInt32) param$U32$2
	ProcCall proc$2
	ProcNoop
Main:
	ProcCall proc$2
Calculating liveness... 0
Calculating liveness for block repeat_merge#4
Old live_in: set()
Old live_out: set()
New live_in: set()
New live_out: set()

Calculating liveness for block repeat_tail#3
Old live_in: set()
Old live_out: set()
Instruction(sets set(), uses {(UInt32) param$U32$2, (UInt32) param$U32$1})
I

In [4]:
G.cfg.pprint()

Control Flow Graph
Block: main#0
	Instructions:
	Predecessors: []
	Successors: ['repeat_head#1']
	Live In: set()
	Live Out: set()

Block: repeat_head#1
	Instructions:
		Instruction(sets {(UInt32) param$U32$1}, uses set())
		Instruction(sets {(UInt32) param$U32$2}, uses set())
	Predecessors: ['repeat_tail#3', 'main#0']
	Successors: ['repeat_body#2']
	Live In: set()
	Live Out: set()

Block: repeat_body#2
	Instructions:
		Instruction(sets {(UInt32) param$U32$3}, uses set())
		Instruction(sets {(UInt32) param$U32$3}, uses {(UInt32) param$U32$3})
		Instruction(sets set(), uses {(UInt32) param$U32$3})
		Instruction(sets {(UInt32) param$U32$4}, uses set())
		Instruction(sets {(UInt32) param$U32$4}, uses {(UInt32) param$U32$4})
		Instruction(sets set(), uses {(UInt32) param$U32$4})
	Predecessors: ['repeat_head#1']
	Successors: ['repeat_tail#3']
	Live In: set()
	Live Out: set()

Block: repeat_tail#3
	Instructions:
		Instruction(sets {(UInt32) param$U32$1}, uses {(UInt32) param$U32$1})
		Instruc

In [5]:
g = G.cfg.calc_liveness()

Calculating liveness... 0
Calculating liveness for block repeat_merge#4
Old live_in: set()
Old live_out: set()
New live_in: set()
New live_out: set()

Calculating liveness for block repeat_tail#3
Old live_in: set()
Old live_out: set()
Instruction(sets set(), uses {(UInt32) param$U32$1, (UInt32) param$U32$2})
Instruction(sets {(UInt32) param$U32$1}, uses {(UInt32) param$U32$1})
New live_in: {(UInt32) param$U32$1, (UInt32) param$U32$2}
New live_out: set()

Calculating liveness for block repeat_body#2
Old live_in: set()
Old live_out: set()
Instruction(sets set(), uses {(UInt32) param$U32$4})
Instruction(sets {(UInt32) param$U32$4}, uses {(UInt32) param$U32$4})
Instruction(sets {(UInt32) param$U32$4}, uses set())
Instruction(sets set(), uses {(UInt32) param$U32$3})
Instruction(sets {(UInt32) param$U32$3}, uses {(UInt32) param$U32$3})
Instruction(sets {(UInt32) param$U32$3}, uses set())
New live_in: {(UInt32) param$U32$2, (UInt32) param$U32$1}
New live_out: {(UInt32) param$U32$1, (UInt32) p

In [6]:
g

{'U32': {(UInt32) param$U32$1: 0,
  (UInt32) param$U32$2: 1,
  (UInt32) param$U32$3: 2,
  (UInt32) param$U32$4: 2}}

In [7]:
from proc_comp.common.csh import ParamRef
param_map = dict()
for ty, color_map in g.items():
    for param, i in color_map.items():
        ref = ParamRef('__reg_'+ty, i)
        param_map[param] = ref

param_map

{(UInt32) param$U32$1: __reg_U32[0],
 (UInt32) param$U32$2: __reg_U32[1],
 (UInt32) param$U32$3: __reg_U32[2],
 (UInt32) param$U32$4: __reg_U32[2]}

In [8]:
G.procedures

{'proc$1': [ProcSet gpio_mode[16] (U8) 1,
  ProcSet gpio_value[16] (U8) 1,
  ProcUnop time idt (UInt32) param$U32$3,
  ProcBinop (UInt32) param$U32$3 + (U32) 1 (UInt32) param$U32$3,
  ProcBlock time >= (UInt32) param$U32$3,
  ProcSet gpio_mode[16] (U8) 1,
  ProcSet gpio_value[16] (U8) 0,
  ProcUnop time idt (UInt32) param$U32$4,
  ProcBinop (UInt32) param$U32$4 + (U32) 1 (UInt32) param$U32$4,
  ProcBlock time >= (UInt32) param$U32$4],
 'proc$2': [ProcSet (UInt32) param$U32$1 (U32) 0,
  ProcSet (UInt32) param$U32$2 (U32) 10,
  ProcCall proc$1,
  ProcUnop (UInt32) param$U32$1 ++ (UInt32) param$U32$1,
  ProcIfElse (UInt32) param$U32$1 < (UInt32) param$U32$2,
  ProcCall proc$2,
  ProcNoop]}

In [16]:
import itertools

from proc_comp.common.csh import CSH_Command

list(
itertools.chain(
    G.main,
    itertools.chain.from_iterable(
        G.procedures.values()
    )
)
)

[ProcCall proc$2,
 ProcSet gpio_mode[16] (U8) 1,
 ProcSet gpio_value[16] (U8) 1,
 ProcUnop time idt (UInt32) param$U32$3,
 ProcBinop (UInt32) param$U32$3 + (U32) 1 (UInt32) param$U32$3,
 ProcBlock time >= (UInt32) param$U32$3,
 ProcSet gpio_mode[16] (U8) 1,
 ProcSet gpio_value[16] (U8) 0,
 ProcUnop time idt (UInt32) param$U32$4,
 ProcBinop (UInt32) param$U32$4 + (U32) 1 (UInt32) param$U32$4,
 ProcBlock time >= (UInt32) param$U32$4,
 ProcSet (UInt32) param$U32$1 (U32) 0,
 ProcSet (UInt32) param$U32$2 (U32) 10,
 ProcCall proc$1,
 ProcUnop (UInt32) param$U32$1 ++ (UInt32) param$U32$1,
 ProcIfElse (UInt32) param$U32$1 < (UInt32) param$U32$2,
 ProcCall proc$2,
 ProcNoop]

In [18]:
for cmd in itertools.chain(G.main,
                            itertools.chain.from_iterable(
                                G.procedures.values()
                            )):
    cmd.update_params(param_map)

G.procedures

{'proc$1': [ProcSet gpio_mode[16] (U8) 1,
  ProcSet gpio_value[16] (U8) 1,
  ProcUnop time idt __reg_U32[2],
  ProcBinop __reg_U32[2] + (U32) 1 __reg_U32[2],
  ProcBlock time >= __reg_U32[2],
  ProcSet gpio_mode[16] (U8) 1,
  ProcSet gpio_value[16] (U8) 0,
  ProcUnop time idt __reg_U32[2],
  ProcBinop __reg_U32[2] + (U32) 1 __reg_U32[2],
  ProcBlock time >= __reg_U32[2]],
 'proc$2': [ProcSet __reg_U32[0] (U32) 0,
  ProcSet __reg_U32[1] (U32) 10,
  ProcCall proc$1,
  ProcUnop __reg_U32[0] ++ __reg_U32[0],
  ProcIfElse __reg_U32[0] < __reg_U32[1],
  ProcCall proc$2,
  ProcNoop]}

In [ ]:
d = dict()

d['a'] = "b"

list(d.values())

['b']

In [ ]:
list(g['U32'].edges)

[((UInt32) param$U32$2, (UInt32) param$U32$1),
 ((UInt32) param$U32$2, (UInt32) param$U32$3),
 ((UInt32) param$U32$2, (UInt32) param$U32$4),
 ((UInt32) param$U32$1, (UInt32) param$U32$3),
 ((UInt32) param$U32$1, (UInt32) param$U32$4)]

In [ ]:
import networkx as nx

G = g['U32']

nx.coloring.greedy_color(G, nx.algorithms.coloring.strategy_saturation_largest_first)

{(UInt32) param$U32$2: 0,
 (UInt32) param$U32$1: 1,
 (UInt32) param$U32$3: 2,
 (UInt32) param$U32$4: 2}

In [ ]:
for b in G.cfg.blocks:
    print("###", b)
    g = G.cfg.calc_liveness_local(b)
    g.pprint()
    print()

AttributeError: 'Graph' object has no attribute 'cfg'

In [ ]:
import types

class Ty:
    __match_args__ = ("a", "b")
    def __init__(self, a, b):
        self.a = a
        self.b = b

t = Ty(1, 1)

match t:
    case Ty(isinstance(a, types.int), isinstance(b, types.int)):
        print("ints")
    case Ty(isinstance(a, types.int), _):
        print("Only a is int")
    case Ty(_, isinstance(b, types.int)):
        print("Only b is int")
    case _:
        print("neither int")

TypeError: called match pattern must be a type

json -parser-> AbSyn -codegen-> ISH 

In [ ]:
from proc_comp.common.cfgbuilder import ControlFlowGraph

cfg = ControlFlowGraph()

b = cfg.block_start("entry")

print(b)
print(cfg.current_block)
print(cfg.blocks)

Block start:  entry Succeeds:  None
Block(entry, set())
Block(entry, set())
[]


In [ ]:
import itertools

s = set([1,2,3,4])

list(itertools.combinations(s, 2))

[(1, 2), (1, 3), (1, 4), (2, 3), (2, 4), (3, 4)]